### run evaluations on checklists of week 4 and week 5 versions

In [1]:
from analyze import *
from modules.llm_eval.consistency_eval import *

In [2]:
report_output_path_html = '../../report/evaluation_report.html'
report_output_path_pdf = '../../report/evaluation_report.pdf'

In [3]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
repo = Repository('../../data/raw/openja/lightfm')
prompt_format = EvaluationPromptFormat()

checklist_v1 = Checklist('archive/checklist_20240527.csv', checklist_format=ChecklistFormat.CSV)
checklist_v2 = Checklist('../../checklist/checklist.csv', checklist_format=ChecklistFormat.CSV)

evaluator_v1 = PerFileTestEvaluator(llm, prompt_format=prompt_format, repository=repo, checklist=checklist_v1)
evaluator_v2 = PerFileTestEvaluator(llm, prompt_format=prompt_format, repository=repo, checklist=checklist_v2)

In [ ]:
eval_evaluator = ConsistencyEvaluator()

RUNS = 20 # TODO: recommended 20 ~ 30

eval_evaluator.evaluate(models=[
    {'name': '1. original', 'model': evaluator_v1},
    {'name': '2. after rewording Requirement', 'model': evaluator_v2},
], num_test_runs=RUNS) 

  0%|                                                                                                                                                                                                                   | 0/7 [00:00<?, ?it/s]

In [ ]:
eval_evaluator.get_completeness_score_dist()

### compare consistency 

In [ ]:
completeness_score_var = pd.DataFrame(eval_evaluator.get_completeness_score_dist().var(axis=1), columns=['sample variance'])
completeness_score_var

In [ ]:
F = completeness_score_var['sample variance'].iloc[1] / completeness_score_var['sample variance'].iloc[0]
F

In [ ]:
import scipy
tail = 2
alpha = 0.05 #Or whatever you want your alpha to be.
p_value = 1 - scipy.stats.f.cdf(F, RUNS-1, RUNS-1) 

print(f"p-value: {p_value}")
print()

print(f"{tail}-tail test:")
if p_value < alpha / 2:
    print("  Reject the null hypothesis: Var(Completeness_Score(Current Version)) == Var(Completeness_Score(Last Week Version))")
else:
    print("  Failed to reject the null hypothesis: Var(Completeness_Score(Current Version)) == Var(Completeness_Score(Last Week Version))")

### compare accuracy

In [ ]:
# Human evaluated answer
human_answer = pd.DataFrame([
    {'ID': '2.1', "human_answer": 1.0}, 
    {'ID': '3.2', "human_answer": 0.5},
    {'ID': '3.5', "human_answer": 0.0},
    {'ID': '4.2', "human_answer": 1.0},
    {'ID': '5.3', "human_answer": 0.5},
    {'ID': '6.1', "human_answer": 1.0},
    {'ID': '6.2', "human_answer": 1.0},
])
# reference: https://github.com/UBC-MDS/test-creation/blob/main/report/repo_human_evaluation/human_evaluation_report-lightfm.md

In [ ]:
df = eval_evaluator.get_consistency_dist().reset_index().drop(columns=['consistency'])
df = df.merge(human_answer, how='left', on='ID')

checklist_item_title = pd.DataFrame(checklist_v2.get_all_tests())[['ID', 'Title']]
df = df.merge(checklist_item_title, how='left', on='ID')
df

### reference
https://www.itl.nist.gov/div898/handbook/eda/section3/eda359.html  
https://www.statisticshowto.com/probability-and-statistics/hypothesis-testing/f-test/#:~:text=F%20Test%20to%20Compare%20Two%20Variances,-A%20Statistical%20F&text=If%20the%20variances%20are%20equal,when%20running%20an%20F%20Test  
https://stackoverflow.com/questions/21494141/how-do-i-do-a-f-test-in-python